In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
df = pd.read_csv("regularite-mensuelle-tgv-aqst.csv",encoding="latin-1")

### Overview

In [3]:
df.columns

Index(['date', 'service', 'gare_depart', 'gare_arrivee', 'duree_moyenne',
       'nb_train_prevu', 'nb_annulation', 'commentaire_annulation',
       'nb_train_depart_retard', 'retard_moyen_depart',
       'retard_moyen_tous_trains_depart', 'commentaire_retards_depart',
       'nb_train_retard_arrivee', 'retard_moyen_arrivee',
       'retard_moyen_tous_trains_arrivee', 'commentaires_retard_arrivee',
       'nb_train_retard_sup_15', 'retard_moyen_trains_retard_sup15',
       'nb_train_retard_sup_30', 'nb_train_retard_sup_60',
       'prct_cause_externe', 'prct_cause_infra', 'prct_cause_gestion_trafic',
       'prct_cause_materiel_roulant', 'prct_cause_gestion_gare',
       'prct_cause_prise_en_charge_voyageurs'],
      dtype='object')

In [32]:
df.head()

,date,service,gare_depart,gare_arrivee,duree_moyenne,nb_train_prevu,nb_annulation,commentaire_annulation,nb_train_depart_retard,retard_moyen_depart,...,nb_train_retard_sup_15,retard_moyen_trains_retard_sup15,nb_train_retard_sup_30,nb_train_retard_sup_60,prct_cause_externe,prct_cause_infra,prct_cause_gestion_trafic,prct_cause_materiel_roulant,prct_cause_gestion_gare,prct_cause_prise_en_charge_voyageurs
0,1/1/2018,National,BORDEAUX ST JEAN,PARIS MONTPARNASSE,141,870,5,NaN,289,11.247809,...,110,6.511118,44,8,36.134454,31.092437,10.924370,15.966387,5.042017,0.840336
1,1/1/2018,National,LA ROCHELLE VILLE,PARIS MONTPARNASSE,165,222,0,NaN,8,2.875000,...,22,5.696096,5,0,15.384615,30.769231,38.461538,11.538462,3.846154,0.000000
2,1/1/2018,National,PARIS MONTPARNASSE,QUIMPER,220,248,1,NaN,37,9.501351,...,26,7.548387,17,7,26.923077,38.461538,15.384615,19.230769,0.000000,0.000000
3,1/1/2018,National,PARIS MONTPARNASSE,ST MALO,156,102,0,NaN,12,19.912500,...,8,6.724757,6,4,23.076923,46.153846,7.692308,15.384615,7.692308,0.000000
4,1/1/2018,National,PARIS MONTPARNASSE,ST PIERRE DES CORPS,61,391,2,NaN,61,7.796995,...,17,3.346487,6,0,21.212121,42.424242,9.090909,21.212121,6.060606,0.000000


In [5]:
(df.isna().sum()/df.shape[0]).sort_values(ascending=True)  # Nan only in the comment columns

date                                    0.000000
prct_cause_materiel_roulant             0.000000
prct_cause_gestion_trafic               0.000000
prct_cause_infra                        0.000000
prct_cause_externe                      0.000000
nb_train_retard_sup_60                  0.000000
nb_train_retard_sup_30                  0.000000
retard_moyen_trains_retard_sup15        0.000000
nb_train_retard_sup_15                  0.000000
retard_moyen_tous_trains_arrivee        0.000000
retard_moyen_arrivee                    0.000000
prct_cause_gestion_gare                 0.000000
nb_train_retard_arrivee                 0.000000
retard_moyen_tous_trains_depart         0.000000
retard_moyen_depart                     0.000000
nb_train_depart_retard                  0.000000
nb_annulation                           0.000000
nb_train_prevu                          0.000000
duree_moyenne                           0.000000
gare_arrivee                            0.000000
gare_depart         

In [6]:
df[df.duplicated()]  # No duplicated rows

,date,service,gare_depart,gare_arrivee,duree_moyenne,nb_train_prevu,nb_annulation,commentaire_annulation,nb_train_depart_retard,retard_moyen_depart,...,nb_train_retard_sup_15,retard_moyen_trains_retard_sup15,nb_train_retard_sup_30,nb_train_retard_sup_60,prct_cause_externe,prct_cause_infra,prct_cause_gestion_trafic,prct_cause_materiel_roulant,prct_cause_gestion_gare,prct_cause_prise_en_charge_voyageurs


### Remove non-predictible features/check for errors

In [3]:
columns_cause = ['prct_cause_externe', 'prct_cause_infra', 'prct_cause_gestion_trafic',
       'prct_cause_materiel_roulant', 'prct_cause_gestion_gare',
       'prct_cause_prise_en_charge_voyageurs']

columns_retard = ['retard_moyen_depart',
       'retard_moyen_tous_trains_depart', 'commentaire_retards_depart',
       'nb_train_retard_arrivee','retard_moyen_tous_trains_arrivee', 'commentaires_retard_arrivee',
       'nb_train_retard_sup_15', 'retard_moyen_trains_retard_sup15',
       'nb_train_retard_sup_30', 'nb_train_retard_sup_60',"nb_train_depart_retard"]

other_columns = ['nb_annulation', 'commentaire_annulation','duree_moyenne']

# "duree_moyenne" could be a useful feature with a few feature engineering 
# (estimating the mean for every line and add the value in a new feature)
# This could also be done with the "retard" features, but there are highly correlated to the target
# Maybe this could be done for the "cause" features?

def clean_dataset(df,other_columns,columns_retard):
    columns_to_remove = other_columns + columns_retard
    df = df.drop(columns_to_remove, axis=1)
    df = df.drop([2886,2889],axis = 0)  # Remove outliers isolated in the next cell
    df = df.reset_index(drop = True)
    for i, d in enumerate(df["date"].tolist()):  # Remove first lockdown
        month, day, year = d.split('/')
        if (int(year) == 2020 and int(month) in [3,4,5]):
              df = df.drop(i,axis = 0)
    return df

def check_errors(df,columns_cause):
    print("number of non-plausible values:")
    print(len(df[df["duree_moyenne"]<0]))
    print(len(df[df["nb_train_prevu"]<0]))
    print(len(df[df["retard_moyen_arrivee"]<0]))
    for col in columns_cause:
       print(len(df[(df[col]<0) | (df[col]>100)]))

check_errors(df,columns_cause)       
clean_df = clean_dataset(df,other_columns,columns_retard,)  # Test
# Dataset cleaning should be done on train and test set separately -> ensure reproducibility


number of non-plausible values:
0
0
2
0
0
0
0
0
0


In [41]:
df[df["retard_moyen_arrivee"]<0] #  Something happened this month?

# I dont' see any valuable reason that would explain these outlier, we can remove or impute them
# TODO check covid period and try impute the previous outlier

,date,service,gare_depart,gare_arrivee,duree_moyenne,nb_train_prevu,nb_annulation,commentaire_annulation,nb_train_depart_retard,retard_moyen_depart,...,nb_train_retard_sup_15,retard_moyen_trains_retard_sup15,nb_train_retard_sup_30,nb_train_retard_sup_60,prct_cause_externe,prct_cause_infra,prct_cause_gestion_trafic,prct_cause_materiel_roulant,prct_cause_gestion_gare,prct_cause_prise_en_charge_voyageurs
2886,11/1/2019,National,MONTPELLIER,PARIS LYON,380,227,11,NaN,189,4.910406,...,44,34.677381,18,3,52.272727,13.636364,15.909091,15.909091,2.272727,0.00000
2889,11/1/2019,National,NIMES,PARIS LYON,224,226,11,NaN,190,8.765614,...,44,34.677381,18,3,46.774194,17.741935,14.516129,12.903226,3.225806,4.83871


### Train-test split

In [4]:
def get_train_test_set(df):
    train_idx = []
    test_idx = []
    for i, d in enumerate(df["date"].tolist()):
        month, day, year = d.split('/')
        if int(year)<2023:
            train_idx.append(i)
        else:
            test_idx.append(i)
    
    train_set = df.iloc[train_idx].copy(deep=True)
    test_set = df.iloc[test_idx].copy(deep=True)
    
    return train_set, test_set

trainset, testset = get_train_test_set(clean_df)

### Preprocessing (encoding/scaling)

In [11]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder,LabelBinarizer,OrdinalEncoder,MinMaxScaler,Normalizer,RobustScaler
from sklearn.feature_extraction import FeatureHasher

def manage_date_column(df):
    df['date'] = pd.to_datetime(df['date'])
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month.apply(str).apply(lambda x:[x])
    df = df.drop("date",axis=1)
    return df

def preprocessing(df1,target,estimated_retard_moyen = False):
    df = df1.copy()
    df = manage_date_column(df)
    df["gare_arrivee"] = df["gare_arrivee"].apply(lambda x:[x])
    df["gare_depart"] = df["gare_depart"].apply(lambda x:[x])
    
    df["ligne"] = df.apply(lambda x:x["gare_arrivee"]+x["gare_depart"],axis = 1)
    df = df.drop(["gare_arrivee","gare_depart"],axis = 1)
    
    scaling_cols = ["nb_train_prevu","year"]  # year could be removed
    hash_cols1 = "month"
    hash_cols2 = ["ligne"]  # "gare_depart","gare_arrivee"
    binarizer_cols = []
    onehot_cols = ["service"]
    ordinal_encode_cols = []
        
    if estimated_retard_moyen:
        scaling_cols += ["estimated_retard_moyen"]
        
    y = df[target] 
    X = df.drop(target,axis = 1)
           
    binarizer_transformer = Pipeline(steps=[
        ('binarizer',LabelBinarizer())])
    hash_transformer1 = Pipeline(steps=[
        ('hashing', FeatureHasher(n_features=4,input_type = "string"))]) # For month
    hash_transformer2 = Pipeline(steps=[
        ('hashing2', FeatureHasher(n_features=16,input_type = "string"))]) # For stations
    numeric_transformer = Pipeline(steps=[
        ('scaler', RobustScaler())])
    onehot_transformer = Pipeline(steps=[
        ('onehot', OneHotEncoder())])
    ordinal_encode_transformer = Pipeline(steps=[
        ('ordinal_encode', OrdinalEncoder())])

    transformers=[
            # ('cat', binarizer_transformer,binarizer_cols),
            ('hash', hash_transformer1, hash_cols1)]
    
    for i in range(len(hash_cols2)):
        transformers.append(('hash'+str(i), hash_transformer2, hash_cols2[i]))
     
    transformers += [('num', numeric_transformer, scaling_cols),
            ('one', onehot_transformer, onehot_cols),
            # ('ord', ordinal_encode_transformer, ordinal_encode_cols)
            ]

    # print(transformers) 
    preprocessor = ColumnTransformer(
        transformers=transformers
        #remainder = 'passthrough', # Will cause undesirerable columns to stay in X_transformed
        )
    
    X_transformed = preprocessor.fit_transform(X).todense()
    
    return X_transformed,y

target = "retard_moyen_arrivee"
# ['prct_cause_externe', 'prct_cause_infra', 'prct_cause_gestion_trafic','prct_cause_materiel_roulant', 'prct_cause_gestion_gare','prct_cause_prise_en_charge_voyageurs']

X_train,y_train = preprocessing(trainset,target)
X_test,y_test = preprocessing(testset,target)


In [12]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error,mean_absolute_error

model = LinearRegression()
model.fit(np.asarray(X_train),np.asarray(y_train))
y_pred = model.predict(np.asarray(X_test))

print(mean_absolute_error(y_pred,y_test))
print(mean_squared_error(y_test,y_pred))

10.599525212681879
344.8618134030007
